In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
training_data = datasets.FashionMNIST(
    root='data',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

In [4]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size = batch_size )
test_dataloader = DataLoader(test_data, batch_size = batch_size)

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10))
        
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch %100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    #모든 img correctness
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298873  [    0/60000]
loss: 2.281952  [ 6400/60000]
loss: 2.258960  [12800/60000]
loss: 2.252796  [19200/60000]
loss: 2.254041  [25600/60000]
loss: 2.202168  [32000/60000]
loss: 2.218419  [38400/60000]
loss: 2.182618  [44800/60000]
loss: 2.172924  [51200/60000]
loss: 2.142123  [57600/60000]
Test Error: 
 Accuracy: 46.0%, Avg loss: 2.135088 

Epoch 2
-------------------------------
loss: 2.141484  [    0/60000]
loss: 2.129702  [ 6400/60000]
loss: 2.069159  [12800/60000]
loss: 2.086555  [19200/60000]
loss: 2.045918  [25600/60000]
loss: 1.970744  [32000/60000]
loss: 1.997345  [38400/60000]
loss: 1.921522  [44800/60000]
loss: 1.908983  [51200/60000]
loss: 1.839411  [57600/60000]
Test Error: 
 Accuracy: 59.9%, Avg loss: 1.837612 

Epoch 3
-------------------------------
loss: 1.865090  [    0/60000]
loss: 1.836393  [ 6400/60000]
loss: 1.718349  [12800/60000]
loss: 1.759454  [19200/60000]
loss: 1.664657  [25600/60000]
loss: 1.609278  [32000/600